<a href="https://colab.research.google.com/github/JeannaWeigandt/my-new-project/blob/main/chatgpt_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
client = OpenAI(api_key = "api-id")

In [ ]:
client

In [ ]:
assistant = client.beta.assistants.create(
  name="demo_assistant_march",
  instructions="You are a personal bot who says funny things.",
  tools=[{"type": "retrieval"}],
  model="gpt-4-turbo-preview",
  file_ids = ["file-W0PqbSBjfZX6vgVT67XaMv4n"]
)

In [ ]:
# uploaded_file = client.files.create(
#     file=open("/content/file.html", 'rb'),
#     purpose='assistants',
# )

In [ ]:
assistant

Assistant(id='asst_1xLPxnT1lg4epqQCEY487T6z', created_at=1711122197, description=None, file_ids=['file-W0PqbSBjfZX6vgVT67XaMv4n'], instructions='You are a personal bot who says funny things.', metadata={}, model='gpt-4-turbo-preview', name='demo_assistant_march', object='assistant', tools=[RetrievalTool(type='retrieval')])

In [ ]:
thread = client.beta.threads.create()

In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="When is the event happening?"
)

In [ ]:
from typing_extensions import override
from openai import AssistantEventHandler

# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.

class EventHandler(AssistantEventHandler):
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)

  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)

  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)

  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)

# Then, we use the `create_and_stream` SDK helper
# with the `EventHandler` class to create the Run
# and stream the response.

with client.beta.threads.runs.create_and_stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address query",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > retrieval


assistant > The event, "AWS Certified Machine Learning - Specialty - Workshop," is happening on Sunday, March 24, 2024, starting at 5:00 PM【5†source】.None